In [1]:
import pandas as pd
import numpy as np

import time
import pickle
from pathlib import Path

In [2]:
import black
import jupyter_black

jupyter_black.load(line_length=79)

In [3]:
# FIX VSCODE PATH PROBLEM -- DELETE AFTER
import sys

# add path
sys.path.append(
    "/gpfs01/berens/user/rgonzalesmarquez/phd/chatgpt-excess-words/scripts/"
)
print(sys.path)

['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/gpfs01/berens/user/rgonzalesmarquez/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/gpfs01/berens/user/rgonzalesmarquez/phd/chatgpt-excess-words/scripts/']


In [5]:
%load_ext autoreload
%autoreload 2

from process_pubmed_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
variables_path = Path("../results/variables")
figures_path = Path("../figures")
data_path = Path("../data")
berenslab_data_path = Path("/gpfs01/berens/data/data/pubmed_processed")

In [111]:
print(variables_path)

/gpfs01/berens/user/rgonzalesmarquez/phd/chatgpt-excess-words/results/variables


In [7]:
# MANUAL FIX TO PATH ISSUE FROM VSCODE

nb_path = Path(
    "/gpfs01/berens/user/rgonzalesmarquez/phd/chatgpt-excess-words/scripts"
)
assert nb_path.exists(), "The path does not exist"

variables_path = (nb_path / variables_path).resolve(strict=True)
figures_path = (nb_path / figures_path).resolve(strict=True)
data_path = (nb_path / data_path).resolve(strict=True)

We extract from PubMed's metadata `.xml` files the following:
- PubMed ID
- title
- abstract
- language
- journal title
- ISSN
- publication date
- (first and last) author first names
- (first and last) author Affiliations


# Parse data

## 2025 baseline

In [ ]:
%%time

path = "/gpfs01/berens/data/data/pubmed/2025_baseline/"

files_2025_df = import_all_files(path, order_files=True)

# save results
files_2025_df.to_pickle(
    "/gpfs01/berens/data/data/pubmed_processed/files_2025_df"
)

pubmed25n0001.xml
pubmed25n0002.xml
pubmed25n0003.xml
pubmed25n0004.xml
pubmed25n0005.xml
pubmed25n0006.xml
pubmed25n0007.xml
pubmed25n0008.xml
pubmed25n0009.xml
pubmed25n0010.xml
pubmed25n0011.xml
pubmed25n0012.xml
pubmed25n0013.xml
pubmed25n0014.xml
pubmed25n0015.xml
pubmed25n0016.xml
pubmed25n0017.xml
pubmed25n0018.xml
pubmed25n0019.xml
pubmed25n0020.xml
pubmed25n0021.xml
pubmed25n0022.xml
pubmed25n0023.xml
pubmed25n0024.xml
pubmed25n0025.xml
pubmed25n0026.xml
pubmed25n0027.xml
pubmed25n0028.xml
pubmed25n0029.xml
pubmed25n0030.xml
pubmed25n0031.xml
pubmed25n0032.xml
pubmed25n0033.xml
pubmed25n0034.xml
pubmed25n0035.xml
pubmed25n0036.xml
pubmed25n0037.xml
pubmed25n0038.xml
pubmed25n0039.xml
pubmed25n0040.xml
pubmed25n0041.xml
pubmed25n0042.xml
pubmed25n0043.xml
pubmed25n0044.xml
pubmed25n0045.xml
pubmed25n0046.xml
pubmed25n0047.xml
pubmed25n0048.xml
pubmed25n0049.xml
pubmed25n0050.xml
pubmed25n0051.xml
pubmed25n0052.xml
pubmed25n0053.xml
pubmed25n0054.xml
pubmed25n0055.xml
pubmed25n0

In [11]:
print("There are {} papers".format(files_2025_df.shape[0]))

There are 38201553 papers


In [12]:
files_2025_df.head()

,PMID,Title,AbstractText,Language,Journal,Date,NameFirstAuthor,NameLastAuthor,ISSN,AffiliationFirstAuthor,AffiliationLastAuthor,filename
0,1,Formate assay in body fluids: application in m...,,eng,Biochemical medicine,1975 Jun,A B,T R,0006-2944,,,pubmed25n0001.xml
1,2,Delineation of the intimate details of the bac...,,eng,Biochemical and biophysical research communica...,1975 Oct 27,K S,R H,1090-2104,,,pubmed25n0001.xml
2,3,Metal substitutions incarbonic anhydrase: a ha...,,eng,Biochemical and biophysical research communica...,1975 Oct 27,R J,R G,0006-291X,,,pubmed25n0001.xml
3,4,Effect of chloroquine on cultured fibroblasts:...,,eng,Biochemical and biophysical research communica...,1975 Oct 27,U N,N N,1090-2104,,,pubmed25n0001.xml
4,5,Atomic models for the polypeptide backbones of...,,eng,Biochemical and biophysical research communica...,1975 Oct 27,W A,K B,1090-2104,,,pubmed25n0001.xml


In [10]:
%%time
files_2025_df = pd.read_pickle(
    "/gpfs01/berens/data/data/pubmed_processed/files_2025_df"
)

CPU times: user 1min 6s, sys: 42.4 s, total: 1min 48s
Wall time: 1min 49s


# Filter

Filter out:
- non-English papers
- papers with empty abstracts
- papers with abstracts shorter than 250 or longer than 4000 symbols
- papers with unfinished abstracts

## Empty abstracts and non-english papers

In [13]:
%%time

print("Before, there were {} papers".format(files_2025_df.shape[0]))

# Eliminate empty abstracts
clean_2025_df = files_2025_df[files_2025_df.AbstractText != ""]

print(
    "After eliminating empty abstracts, there are {} papers".format(
        clean_2025_df.shape[0]
    )
)

# Eliminate non-english papers
clean_2025_df = clean_2025_df[clean_2025_df.Language == "eng"]

# print size
print(
    "After first cleaning, there are {} papers".format(clean_2025_df.shape[0])
)

Before, there were 38201553 papers
After eliminating empty abstracts, there are 26748681 papers
After first cleaning, there are 25122472 papers
CPU times: user 19.6 s, sys: 3.56 s, total: 23.2 s
Wall time: 23.2 s


## Threshold and cut off

### Cut off = 4000

In [14]:
print("Before cut off, there are {} papers".format(clean_2025_df.shape[0]))
abstracts = clean_2025_df["AbstractText"].tolist()
len_strings = map(len, abstracts)
len_abstracts = np.fromiter(len_strings, dtype=np.int64, count=len(abstracts))


cut_off = 4000
clean_2025_df = clean_2025_df[len_abstracts < cut_off]
print("After cut off, there are {} papers".format(clean_2025_df.shape[0]))

Before cut off, there are 25122472 papers
After cut off, there are 25103731 papers


### Threshold = 250

In [15]:
threshold = 250
len_short_abstracts = len_abstracts[len_abstracts < cut_off]
clean_2025_df = clean_2025_df[len_short_abstracts > threshold]
print("After threshold, there are {} papers".format(clean_2025_df.shape[0]))

After threshold, there are 24845923 papers


## Remove the truncated sentence from abstracts


In [16]:
abstracts = clean_2025_df["AbstractText"]
abstracts_list = clean_2025_df["AbstractText"].tolist()

In [17]:
%%time

clean_2025_df.AbstractText = list(
    map(
        lambda x, y: x[: y - 1] if y != -1 else x,
        clean_2025_df.AbstractText,
        clean_2025_df.AbstractText.str.find("ABSTRACT TRUNCATED AT"),
    )
)

CPU times: user 29.2 s, sys: 1.69 s, total: 30.9 s
Wall time: 30.9 s


## Remove unfinished abstracts

In [18]:
abstracts = clean_2025_df["AbstractText"]
abstracts_list = clean_2025_df["AbstractText"].tolist()
print(len(abstracts_list))

24845923


In [19]:
%%time
end_abstracts = [x[-2:] for x in abstracts_list]

CPU times: user 4.58 s, sys: 1.06 s, total: 5.64 s
Wall time: 5.66 s


In [20]:
%%time
point_index = np.array([x.find(".") for x in end_abstracts])
question_index = np.array([x.find("?") for x in end_abstracts])
exclamation_index = np.array([x.find("!") for x in end_abstracts])

CPU times: user 9.18 s, sys: 61 ms, total: 9.24 s
Wall time: 9.26 s


In [21]:
%%time

print("Before cleaning, there are {} papers".format(clean_2025_df.shape[0]))

# Eliminate unfinished abstracts
clean_2025_df = clean_2025_df[
    (point_index != -1) | (question_index != -1) | (exclamation_index != -1)
]

# print size
print("After cleaning, there are {} papers".format(clean_2025_df.shape[0]))

Before cleaning, there are 24845923 papers


After cleaning, there are 24817977 papers
CPU times: user 6.79 s, sys: 5.62 s, total: 12.4 s
Wall time: 12.4 s


### Save clean_df

In [81]:
# Fix the issue of one paper having the wronge date that does not satisfy the pattern "([12]\d\d\d)"
# Paper with PMID: 32764183, wrong date: "3276 4183", right date (according to PubMed): "2020"

assert clean_2025_df.PMID.loc[31304337] == "32764183"
assert clean_2025_df.Date.loc[31304337] == "3276 4183"
clean_2025_df.Date.loc[31304337] = "2020"

/tmp/ipykernel_23701/3428046920.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  clean_2025_df.Date.loc[31304337] = "2020"


In [39]:
# Fix the issue of one paper having the wronge date that does not satisfy the pattern "([12]\d\d\d)"
# Paper with PMID: 31151109, wrong date: "3115 1109", right date (according to PubMed): "2019"

assert clean_2025_df.PMID.loc[29734806] == "31151109"
assert clean_2025_df.Date.loc[29734806] == "3115 1109"
clean_2025_df.Date.loc[29734806] = "2019"

In [83]:
clean_2025_df.head()

,PMID,Title,AbstractText,Language,Journal,Date,NameFirstAuthor,NameLastAuthor,ISSN,AffiliationFirstAuthor,AffiliationLastAuthor,filename
22,24,Influence of a new virostatic compound on the ...,"The virostatic compound N,N-diethyl-4-[2-(2-ox...",eng,Arzneimittel-Forschung,1975 Sep,H,G,0004-4172,,,pubmed25n0001.xml
23,23,Effect of etafenone on total and regional myoc...,The distribution of blood flow to the subendoc...,eng,Arzneimittel-Forschung,1975 Sep,H,W,0004-4172,,,pubmed25n0001.xml
24,25,Pharmacological properties of new neuroleptic ...,"RMI 61 140, RMI 61 144 and RMI 61 280 are newl...",eng,Arzneimittel-Forschung,1975 Sep,L,A,0004-4172,,,pubmed25n0001.xml
29,30,Lysosomal hydrolases of the epidermis. I. Glyc...,Seven distinct glycosidases (EC 3.2) have been...,eng,The British journal of dermatology,1975 Jul,P D,J J,0007-0963,,,pubmed25n0001.xml
31,32,A serum haemagglutinating property dependent u...,A serum agglutinin reactive with red cells in ...,eng,British journal of haematology,1975 Jan,M L,W L,0007-1048,,,pubmed25n0001.xml


In [41]:
%%time
# save intermediate dataframe

clean_2025_df.to_pickle(berenslab_data_path / "clean_2025_df_v2")

CPU times: user 1min 35s, sys: 45.5 s, total: 2min 20s
Wall time: 2min 42s


# Label
Generate labels based on the journal title

In [89]:
label_color_legend = {
    "cancer": "black",
    "neuroscience": "#aeaa00",
    "cardiology": "#1CE6FF",
    "ecology": "#FF34FF",
    "bioinformatics": "#FF4A46",
    "chemistry": "#008941",
    "surgery": "#006FA6",
    "environment": "#0089A3",
    "material": "#0000A6",
    "microbiology": "#B79762",
    "pediatric": "#004D43",
    "immunology": "#8FB0FF",
    "psychology": "#5A0007",
    "psychiatry": "#BA0900",
    "genetics": "#1B4400",
    "nutrition": "#4FC601",
    "veterinary": "#3B5DFF",
    "engineering": "#00C2A0",
    "education": "#549E79",
    "physics": "#BC23FF",
    "optics": "#C895C5",
    "nursing": "#FF2F80",
    "neurology": "#009271",
    "radiology": "#00FECF",
    "ophthalmology": "#A4E804",
    "gynecology": "#FFB500",
    "rehabilitation": "#6B002C",
    "pathology": "#FF9408",
    "anesthesiology": "#CC0744",
    "dermatology": "#D790FF",
    "pharmacology": "#5B4534",
    "physiology": "#E83000",
    "virology": "#6F0062",
    "biochemistry": "#b65141",
    "computation": "#C20078",
    "infectious": "#7A4900",
    "healthcare": "#FF90C9",
    "ethics": "#6508ba",
    "dentistry": "#8e4d8a",
}

In [90]:
%%time
labels_2025, _ = improved_coloring(clean_2025_df.Journal, label_color_legend)

CPU times: user 10min 17s, sys: 21.4 s, total: 10min 38s
Wall time: 10min 39s


# Country
Extract first author's affiliation country

In [91]:
all_countries = [
    "Afghanistan",
    "Albania",
    "Algeria",
    "Andorra",
    "Angola",
    "Antigua and Barbuda",
    "Argentina",
    "Armenia",
    "Australia",
    "Austria",
    "Azerbaijan",
    "Bahamas",
    "Bahrain",
    "Bangladesh",
    "Barbados",
    "Belarus",
    "Belgium",
    "Belize",
    "Benin",
    "Bhutan",
    "Bolivia",
    "Bosnia and Herzegovina",
    "Botswana",
    "Brazil",
    "Brunei",
    "Bulgaria",
    "Burkina Faso",
    "Burundi",
    "Cabo Verde",
    "Cambodia",
    "Cameroon",
    "Canada",
    "Central African Republic",
    "Chad",
    "Chile",
    "China",
    "Colombia",
    "Comoros",
    "Democratic Republic of the Congo",
    "Republic of the Congo",
    "Costa Rica",
    "Côte d’Ivoire",
    "Croatia",
    "Cuba",
    "Cyprus",
    "Czech Republic",
    "Denmark",
    "Djibouti",
    "Dominica",
    "Dominican Republic",
    "East Timor",
    "Ecuador",
    "Egypt",
    "El Salvador",
    "Equatorial Guinea",
    "Eritrea",
    "Estonia",
    "Eswatini",
    "Ethiopia",
    "Fiji",
    "Finland",
    "France",
    "Gabon",
    "Gambia",
    "Georgia",
    "Germany",
    "Ghana",
    "Greece",
    "Grenada",
    "Guatemala",
    "Guinea",
    "Guinea-Bissau",
    "Guyana",
    "Haiti",
    "Honduras",
    "Hungary",
    "Iceland",
    "India",
    "Indonesia",
    "Iran",
    "Iraq",
    "Ireland",
    "Israel",
    "Italy",
    "Jamaica",
    "Japan",
    "Jordan",
    "Kazakhstan",
    "Kenya",
    "Kiribati",
    "North Korea",
    "South Korea",
    "Kosovo",
    "Kuwait",
    "Kyrgyzstan",
    "Laos",
    "Latvia",
    "Lebanon",
    "Nicaragua",
    "Niger",
    "Nigeria",
    "North Macedonia",
    "Norway",
    "Oman",
    "Pakistan",
    "Palau",
    "Panama",
    "Papua New Guinea",
    "Paraguay",
    "Peru",
    "Philippines",
    "Poland",
    "Portugal",
    "Qatar",
    "Romania",
    "Russia",
    "Rwanda",
    "Saint Kitts and Nevis",
    "Saint Lucia",
    "Saint Vincent and the Grenadines",
    "Samoa",
    "San Marino",
    "Sao Tome and Principe",
    "Saudi Arabia",
    "Senegal",
    "Serbia",
    "Seychelles",
    "Sierra Leone",
    "Singapore",
    "Slovakia",
    "Slovenia",
    "Solomon Islands",
    "Somalia",
    "South Africa",
    "Spain",
    "Sri Lanka",
    "Sudan",
    "South Sudan",
    "Suriname",
    "Sweden",
    "Switzerland",
    "Syria",
    "Taiwan",
    "Tajikistan",
    "Tanzania",
    "Thailand",
    "Togo",
    "Tonga",
    "Trinidad and Tobago",
    "Tunisia",
    "Turkey",
    "Turkmenistan",
    "Tuvalu",
    "Uganda",
    "Ukraine",
    "United Arab Emirates",
    "United Kingdom",
    "United States",
    "Uruguay",
    "Uzbekistan",
    "Vanuatu",
    "Vatican City",
    "Venezuela",
    "Vietnam",
    "Yemen",
    "Zambia",
    "Zimbabwe",
]

In [92]:
dict_countries = dict(zip(all_countries, np.arange(1, len(all_countries) + 1)))

## Country mapping

In [93]:
%%time
countries_first_author_2025, _ = mapping_countries(
    clean_2025_df.AffiliationFirstAuthor, dict_countries
)

CPU times: user 49min 24s, sys: 51.2 s, total: 50min 15s
Wall time: 50min 17s


## State mapping

In [94]:
all_states = [
    "Alabama",
    "Alaska",
    "Arizona",
    "Arkansas",
    "California",
    "Colorado",
    "Connecticut",
    "Delaware",
    "Florida",
    "Georgia",
    "Hawaii",
    "Idaho",
    "Illinois",
    "Indiana",
    "Iowa",
    "Kansas",
    "Kentucky",
    "Louisiana",
    "Maine",
    "Maryland",
    "Massachusetts",
    "Michigan",
    "Minnesota",
    "Mississippi",
    "Missouri",
    "Montana",
    "Nebraska",
    "Nevada",
    "New Hampshire",
    "New Jersey",
    "New Mexico",
    "New York",
    "North Carolina",
    "North Dakota",
    "Ohio",
    "Oklahoma",
    "Oregon",
    "Pennsylvania",
    "Rhode Island",
    "South Carolina",
    "South Dakota",
    "Tennessee",
    "Texas",
    "Utah",
    "Vermont",
    "Virginia",
    "Washington",
    "West Virginia",
    "Wisconsin",
    "Wyoming",
]

[autoreload of process_pubmed_utils failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.10/dist-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/usr/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/gpfs01/berens/user/rgonzalesmarquez/phd/chatgpt-excess-words/scripts/process_pubmed_utils.py", line 4, in <module>
    import torch
ModuleNotFoundError: No module named 'torch'
]


In [95]:
len(all_states)

50

In [96]:
dict_all_states = dict(zip(all_states, np.arange(1, len(all_states) + 1)))

In [97]:
%%time
labels_usa_states_first_author_2025, _ = mapping_states(
    clean_2025_df.AffiliationFirstAuthor, dict_all_states
)

CPU times: user 14min 18s, sys: 15.2 s, total: 14min 33s
Wall time: 14min 34s


### Correct USA country with states

In [98]:
countries_first_author_2025_usa_corrected = np.where(
    labels_usa_states_first_author_2025 != "unknown",
    "United States",
    countries_first_author_2025,
)

In [99]:
print(
    "Percentage of US papers before correcting",
    np.sum(countries_first_author_2025 == "United States")
    / countries_first_author_2025.shape[0]
    * 100,
)
print(
    "Percentage of US papers after correcting",
    np.sum(countries_first_author_2025_usa_corrected == "United States")
    / countries_first_author_2025.shape[0]
    * 100,
)

Percentage of US papers before correcting 19.12575307810141
Percentage of US papers after correcting 23.899611156864236


# Gender

In this part of the notebook we use the `gender` R package. You can install them in R via comannd `install.packages("gender")`.

In [43]:
# load
clean_2025_df = pd.read_pickle(berenslab_data_path / "clean_2025_df_v2")

## Curate first name lists

### First author

In [44]:
name_first_author = clean_2025_df.NameFirstAuthor

In [45]:
print(
    "Percentge of no names: ",
    np.sum(name_first_author == "") / len(name_first_author) * 100,
)

Percentge of no names:  0.3543157445911083


#### Slicing

In [46]:
%%time
sliced_name_first_author = [
    elem.split()[0] if elem != "" else elem for elem in name_first_author
]

CPU times: user 3.96 s, sys: 114 ms, total: 4.07 s
Wall time: 4.07 s


In [47]:
%%time
sliced_name_first_author = [
    elem.split("-")[0] if elem != "" else elem
    for elem in sliced_name_first_author
]

CPU times: user 2.52 s, sys: 12.7 ms, total: 2.53 s
Wall time: 2.53 s


In [48]:
%%time
len_sliced_name_first_author = [len(x) for x in sliced_name_first_author]

CPU times: user 962 ms, sys: 7.6 ms, total: 970 ms
Wall time: 971 ms


#### Filtering initials

In [49]:
n_initials = len(
    np.array(len_sliced_name_first_author)[
        np.array(len_sliced_name_first_author) == 1
    ]
)
n_total_papers = len(np.array(len_sliced_name_first_author))
print(
    "Percentage of names that are just initials: ",
    n_initials / n_total_papers * 100,
)

Percentage of names that are just initials:  29.3588957713999


In [50]:
print(
    "Percentage of papers with missing names: ",
    np.sum(np.array(sliced_name_first_author) == "")
    / len(sliced_name_first_author)
    * 100,
)

Percentage of papers with missing names:  0.35466629693467766


In [51]:
print(
    "Number of missing names: ",
    len(
        np.array(sliced_name_first_author)[
            np.array(sliced_name_first_author) == ""
        ]
    ),
)

Number of missing names:  88021


In [52]:
%%time
filtered_name_first_author = np.where(
    np.array(len_sliced_name_first_author) == 1, "", sliced_name_first_author
)

CPU times: user 3.44 s, sys: 591 ms, total: 4.03 s
Wall time: 4.03 s


In [53]:
n_initials_and_missing = len(
    np.array(filtered_name_first_author)[
        np.array(filtered_name_first_author) == ""
    ]
)
print(
    "Percentage of initials + missing names: ",
    n_initials_and_missing / n_total_papers * 100,
)

Percentage of initials + missing names:  29.713562068334582


### Last author

In [54]:
name_last_author = clean_2025_df.NameLastAuthor

#### Slicing

In [55]:
%%time
sliced_name_last_author = [
    elem.split()[0] if elem != "" else elem for elem in name_last_author
]

CPU times: user 4.06 s, sys: 211 ms, total: 4.27 s
Wall time: 4.28 s


In [56]:
%%time
sliced_name_last_author = [
    elem.split("-")[0] if elem != "" else elem
    for elem in sliced_name_last_author
]

CPU times: user 2.53 s, sys: 10.7 ms, total: 2.54 s
Wall time: 2.55 s


In [57]:
%%time
len_sliced_name_last_author = [len(x) for x in sliced_name_last_author]

CPU times: user 996 ms, sys: 6.58 ms, total: 1 s
Wall time: 1 s


#### Filtering initials

In [58]:
n_initials = len(
    np.array(len_sliced_name_last_author)[
        np.array(len_sliced_name_last_author) == 1
    ]
)
n_total_papers = len(np.array(len_sliced_name_last_author))
print(
    "Percentage of names that are just initials: ",
    n_initials / n_total_papers * 100,
)

Percentage of names that are just initials:  29.753424302069426


In [59]:
print(
    "Number of missing names: ",
    len(
        np.array(sliced_name_last_author)[
            np.array(sliced_name_last_author) == ""
        ]
    ),
)

Number of missing names:  282010


In [60]:
%%time
filtered_name_last_author = np.where(
    np.array(len_sliced_name_last_author) == 1, "", sliced_name_last_author
)

CPU times: user 3.6 s, sys: 614 ms, total: 4.21 s
Wall time: 4.21 s


In [61]:
n_initials_and_missing = len(
    np.array(filtered_name_last_author)[
        np.array(filtered_name_last_author) == ""
    ]
)
print(
    "Percentage of initials + missing names: ",
    n_initials_and_missing / n_total_papers * 100,
)

Percentage of initials + missing names:  30.8897377090808


## Gender prediction

In [62]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [63]:
%R require(gender)
%R require(tibble)

1


In [64]:
date_year = clean_2025_df.Date.str.extract("([12]\d\d\d)").values.astype(int).squeeze()

### First author

In [72]:
# I initialize a dataframe where I will store year, name and gender
# This will be the dataframe `year_name_gender_first_author_df'

df_subset = pd.DataFrame(
    {"Years": date_year, "Names": filtered_name_first_author}
)
df_subset

,Years,Names
0,1975,
1,1975,
2,1975,
3,1975,
4,1975,
...,...,...
24817972,2025,Shu
24817973,2025,NokI
24817974,2025,Heriberto
24817975,2025,Shahab


In [74]:
%%time

# I predict the gender of those names sorting the df by year.
# The problem is that the returned df does not have the same dimensions as the original, since not all the papers have names.
# Therefore, the next two steps are necessary

gender_prediction_df = pd.DataFrame()
unique_years = np.unique(df_subset.Years)

for year in unique_years:
    print("Year: ", year)

    df_grouped_year = df_subset.groupby("Years").get_group(year)
    df_names = df_grouped_year.Names

    # cases for years outside interval
    if year <= 1930:
        %R library(gender)
        %R -i df_names -o result result = gender(df_names, years = 1930, method = "ssa")

    if year >= 2012:
        %R library(gender)
        %R -i df_names -o result result = gender(df_names, years = 2012, method = "ssa")

    if (year < 2012) & (year > 1930):
        %R library(gender)
        %R -i df_names -i year -o result result = gender(df_names, years = year, method = "ssa")

    gender_prediction_df = pd.concat(
        [gender_prediction_df, result], ignore_index=True
    )

gender_prediction_df

Year:  1808
Year:  1876
Year:  1881
Year:  1883
Year:  1891
Year:  1896
Year:  1897
Year:  1898
Year:  1899
Year:  1900
Year:  1901
Year:  1902
Year:  1903
Year:  1905
Year:  1906
Year:  1907
Year:  1908
Year:  1909
Year:  1910
Year:  1911
Year:  1912
Year:  1913
Year:  1914
Year:  1915
Year:  1916
Year:  1917
Year:  1918
Year:  1919
Year:  1920
Year:  1921
Year:  1922
Year:  1923
Year:  1924
Year:  1925
Year:  1926
Year:  1927
Year:  1928
Year:  1929
Year:  1930
Year:  1931
Year:  1932
Year:  1933
Year:  1934
Year:  1935
Year:  1936
Year:  1937
Year:  1938
Year:  1939
Year:  1940
Year:  1941
Year:  1942
Year:  1943
Year:  1944
Year:  1945
Year:  1946
Year:  1947
Year:  1948
Year:  1949
Year:  1950
Year:  1951
Year:  1952
Year:  1953
Year:  1954
Year:  1955
Year:  1956
Year:  1957
Year:  1958
Year:  1959
Year:  1960
Year:  1961
Year:  1962
Year:  1963
Year:  1964
Year:  1965
Year:  1966
Year:  1967
Year:  1968
Year:  1969
Year:  1970
Year:  1971
Year:  1972
Year:  1973
Year:  1974
Year

,name,proportion_male,proportion_female,gender,year_min,year_max
0,Chalmers,1.0000,0.0000,male,1930.0,1930.0
1,Edmund,0.9940,0.0060,male,1930.0,1930.0
2,Leigh,0.7342,0.2658,male,1930.0,1930.0
3,Mary,0.0056,0.9944,female,1932.0,1932.0
4,Ben,0.9866,0.0134,male,1933.0,1933.0
...,...,...,...,...,...,...
10533564,Zuzanna,0.0000,1.0000,female,2012.0,2012.0
10533565,Zuzanna,0.0000,1.0000,female,2012.0,2012.0
10533566,Zuzu,0.0000,1.0000,female,2012.0,2012.0
10533567,Zvi,1.0000,0.0000,male,2012.0,2012.0


In [75]:
%%time

# In here I am creating a dictionary that has a map name-gender for every year, based on the predictions from the dataframe above (because they are year dependent).
# I do some tricks to create mappings for also years that were not predicted, and years outside the available `ssa' intervals (<1930, >2012)

unique_predicted_years = np.unique(gender_prediction_df["year_min"])
gender_maps_years = {}
for year in unique_years:
    if year >= 2012:
        eff_year = 2012
    if year <= 1930:
        eff_year = 1930
    if (year < 2012) & (year > 1930):
        eff_year = year

    if eff_year not in unique_predicted_years:
        closest_year = unique_predicted_years[
            (unique_predicted_years - eff_year).argmin()
        ]

        gender_prediction_grouped_year = gender_prediction_df.groupby(
            "year_min"
        ).get_group(closest_year)

    else:
        gender_prediction_grouped_year = gender_prediction_df.groupby(
            "year_min"
        ).get_group(eff_year)

    gender_map = dict(
        zip(
            gender_prediction_grouped_year.name,
            gender_prediction_grouped_year.gender,
        )
    )

    gender_maps_years[year] = gender_map

gender_maps_years.keys()

CPU times: user 50.4 s, sys: 214 ms, total: 50.6 s
Wall time: 50.7 s


dict_keys([np.int64(1808), np.int64(1876), np.int64(1881), np.int64(1883), np.int64(1891), np.int64(1896), np.int64(1897), np.int64(1898), np.int64(1899), np.int64(1900), np.int64(1901), np.int64(1902), np.int64(1903), np.int64(1905), np.int64(1906), np.int64(1907), np.int64(1908), np.int64(1909), np.int64(1910), np.int64(1911), np.int64(1912), np.int64(1913), np.int64(1914), np.int64(1915), np.int64(1916), np.int64(1917), np.int64(1918), np.int64(1919), np.int64(1920), np.int64(1921), np.int64(1922), np.int64(1923), np.int64(1924), np.int64(1925), np.int64(1926), np.int64(1927), np.int64(1928), np.int64(1929), np.int64(1930), np.int64(1931), np.int64(1932), np.int64(1933), np.int64(1934), np.int64(1935), np.int64(1936), np.int64(1937), np.int64(1938), np.int64(1939), np.int64(1940), np.int64(1941), np.int64(1942), np.int64(1943), np.int64(1944), np.int64(1945), np.int64(1946), np.int64(1947), np.int64(1948), np.int64(1949), np.int64(1950), np.int64(1951), np.int64(1952), np.int64(1953

In [76]:
%%time
# Now, using the name-gender maps created above, I map the names in the original dataframe (df_subset) to their respective genders, saving them in a new column 'Gender'

# here I add a 'Gender' column to the df_subset
df_subset["Gender"] = ["unknown"] * df_subset.shape[0]

for year in unique_years:
    print(year)

    df_subset_year = (
        df_subset.groupby("Years")
        .get_group(year)
        .Names.apply(lambda x: np.vectorize(gender_maps_years[year].get)(x))
    )
    df_subset_year.rename("Gender", inplace=True)
    df_subset.update(df_subset_year)

df_subset

1808
1876
1881
1883
1891
1896
1897
1898
1899
1900
1901
1902
1903
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2143
CPU times: user 7min 18s, sys: 1.43 s, total: 7min 19s
Wall time: 7min 19s


,Years,Names,Gender
0,1975,,unknown
1,1975,,unknown
2,1975,,unknown
3,1975,,unknown
4,1975,,unknown
...,...,...,...
24817972,2025,Shu,unknown
24817973,2025,NokI,unknown
24817974,2025,Heriberto,male
24817975,2025,Shahab,male


In [112]:
# check that the number of predicted genders matches the length of the df with the predictions above
len(df_subset[df_subset.Gender != "unknown"])

10624183

In [78]:
#  save
gender_first_author = df_subset.Gender.to_numpy(dtype=str)
np.save(variables_path / "gender_first_author", gender_first_author)

### Last author

In [79]:
df_subset = pd.DataFrame(
    {"Years": date_year, "Names": filtered_name_last_author}
)
df_subset

,Years,Names
0,1975,
1,1975,
2,1975,
3,1975,
4,1975,
...,...,...
24817972,2025,Jian
24817973,2025,Wenli
24817974,2025,Bachir
24817975,2025,Mohammad


In [80]:
%%time

gender_prediction_df = pd.DataFrame()
unique_years = np.unique(df_subset.Years)

for year in unique_years:
    print(year)
    df_grouped_year = df_subset.groupby("Years").get_group(year)
    df_names = df_grouped_year.Names

    # cases for years outside interval
    if year <= 1930:
        %R library(gender)
        %R -i df_names -o result result = gender(df_names, years = 1930, method = "ssa")

    if year >= 2012:
        %R library(gender)
        %R -i df_names -o result result = gender(df_names, years = 2012, method = "ssa")

    if (year < 2012) & (year > 1930):
        %R library(gender)
        %R -i df_names -i year -o result result = gender(df_names, years = year, method = "ssa")

    gender_prediction_df = pd.concat(
        [gender_prediction_df, result], ignore_index=True
    )

gender_prediction_df

1808
1876
1881
1883
1891
1896
1897
1898
1899
1900
1901
1902
1903
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2143
CPU times: user 5min 22s, sys: 3.98 s, total: 5min 26s
Wall time: 5min 26s


,name,proportion_male,proportion_female,gender,year_min,year_max
0,Chalmers,1.0000,0.0000,male,1930.0,1930.0
1,Edmund,0.9940,0.0060,male,1930.0,1930.0
2,Leigh,0.7342,0.2658,male,1930.0,1930.0
3,Mary,0.0056,0.9944,female,1932.0,1932.0
4,Rose,0.0067,0.9933,female,1932.0,1932.0
...,...,...,...,...,...,...
10624178,Zuzana,0.0000,1.0000,female,2012.0,2012.0
10624179,Zuzana,0.0000,1.0000,female,2012.0,2012.0
10624180,Zuzanna,0.0000,1.0000,female,2012.0,2012.0
10624181,Zvi,1.0000,0.0000,male,2012.0,2012.0


In [81]:
%%time

unique_predicted_years = np.unique(gender_prediction_df["year_min"])
gender_maps_years = {}
for year in unique_years:
    if year >= 2012:
        eff_year = 2012
    if year <= 1930:
        eff_year = 1930
    if (year < 2012) & (year > 1930):
        eff_year = year

    if eff_year not in unique_predicted_years:
        closest_year = unique_predicted_years[
            (unique_predicted_years - eff_year).argmin()
        ]

        gender_prediction_grouped_year = gender_prediction_df.groupby(
            "year_min"
        ).get_group(closest_year)

    else:
        gender_prediction_grouped_year = gender_prediction_df.groupby(
            "year_min"
        ).get_group(eff_year)

    gender_map = dict(
        zip(
            gender_prediction_grouped_year.name,
            gender_prediction_grouped_year.gender,
        )
    )

    gender_maps_years[year] = gender_map

gender_maps_years.keys()

CPU times: user 50.9 s, sys: 407 ms, total: 51.4 s
Wall time: 51.4 s


dict_keys([np.int64(1808), np.int64(1876), np.int64(1881), np.int64(1883), np.int64(1891), np.int64(1896), np.int64(1897), np.int64(1898), np.int64(1899), np.int64(1900), np.int64(1901), np.int64(1902), np.int64(1903), np.int64(1905), np.int64(1906), np.int64(1907), np.int64(1908), np.int64(1909), np.int64(1910), np.int64(1911), np.int64(1912), np.int64(1913), np.int64(1914), np.int64(1915), np.int64(1916), np.int64(1917), np.int64(1918), np.int64(1919), np.int64(1920), np.int64(1921), np.int64(1922), np.int64(1923), np.int64(1924), np.int64(1925), np.int64(1926), np.int64(1927), np.int64(1928), np.int64(1929), np.int64(1930), np.int64(1931), np.int64(1932), np.int64(1933), np.int64(1934), np.int64(1935), np.int64(1936), np.int64(1937), np.int64(1938), np.int64(1939), np.int64(1940), np.int64(1941), np.int64(1942), np.int64(1943), np.int64(1944), np.int64(1945), np.int64(1946), np.int64(1947), np.int64(1948), np.int64(1949), np.int64(1950), np.int64(1951), np.int64(1952), np.int64(1953

In [82]:
%%time

df_subset["Gender"] = ["unknown"] * df_subset.shape[0]

for year in unique_years:
    print(year)

    df_subset_year = (
        df_subset.groupby("Years")
        .get_group(year)
        .Names.apply(lambda x: np.vectorize(gender_maps_years[year].get)(x))
    )
    df_subset_year.rename("Gender", inplace=True)
    df_subset.update(df_subset_year)

df_subset

1808
1876
1881
1883
1891
1896
1897
1898
1899
1900
1901
1902
1903
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2143
CPU times: user 7min 18s, sys: 1.27 s, total: 7min 19s
Wall time: 7min 20s


,Years,Names,Gender
0,1975,,unknown
1,1975,,unknown
2,1975,,unknown
3,1975,,unknown
4,1975,,unknown
...,...,...,...
24817972,2025,Jian,male
24817973,2025,Wenli,unknown
24817974,2025,Bachir,unknown
24817975,2025,Mohammad,male


In [113]:
# check that the number of predicted genders matches the length of the df with the predictions above
len(df_subset[df_subset.Gender != "unknown"])

10624183

In [84]:
#  save
gender_last_author = df_subset.Gender.to_numpy(dtype=str)
np.save(variables_path / "gender_last_author_2024", gender_last_author)

# Save dataframe


- PMID
- abstract
- journal
- publication year (update also preprocess-and-count bc the year is being extracted there)
- label
- affiliation country (from the first affiliation of the first author).
- inferred gender of first and last author

In [85]:
df = clean_2025_df[["PMID", "Title", "AbstractText", "Journal", "Date"]].copy(
    deep=True
)

In [86]:
df["Year"] = df.Date.str.extract("([12]\d\d\d)").values.astype(int)

In [87]:
df.pop("Date")

22             1975 Sep
23             1975 Sep
24             1975 Sep
29             1975 Jul
31             1975 Jan
               ...     
38201548    2025 Jan 08
38201549    2025 Jan 08
38201550       2025 Feb
38201551    2025 Jan 07
38201552    2025 Jan 07
Name: Date, Length: 24817977, dtype: object

In [100]:
df["Labels"] = labels_2025

In [101]:
df["Countries"] = countries_first_author_2025_usa_corrected

In [102]:
df["InferredGenderFirstAuthor"] = gender_first_author
df["InferredGenderLastAuthor"] = gender_last_author

In [103]:
len(df)

24817977

In [104]:
df = df.groupby(["PMID"], as_index=False).last()

In [105]:
print(f"There are {len(df)} new papers")

There are 24814136 new papers


In [106]:
df.head()

,PMID,Title,AbstractText,Journal,Year,Labels,Countries,InferredGenderFirstAuthor,InferredGenderLastAuthor
0,1000,The amino acid sequence of Neurospora NADP-spe...,The NADP-specific glutamate dehydrogenase of N...,The Biochemical journal,1975,unlabeled,unknown,unknown,unknown
1,10000,A new method for the determination of alpha1-p...,Up until now it has been assumed that the prot...,Biochimica et biophysica acta,1976,unlabeled,unknown,unknown,unknown
2,1000017,Development of a short term assay for lymphobl...,Blastic stimulation of lymphocytes by antigens...,Biomedicine / [publiee pour l'A.A.I.C.I.G.],1976,unlabeled,unknown,unknown,unknown
3,1000018,Neonatal polycythemia in low birthweight infant.,74 low birthweight infants haematologic findin...,Biomedicine / [publiee pour l'A.A.I.C.I.G.],1976,unlabeled,unknown,unknown,unknown
4,1000019,Is there evidence for subclasses of chronic ly...,Various clinical and biological parameters wer...,Biomedicine / [publiee pour l'A.A.I.C.I.G.],1976,unlabeled,unknown,unknown,unknown


In [107]:
df.shape

(24814136, 9)

In [108]:
%%time
compression_opts = dict(method="zip", archive_name="pubmed_baseline_2025.csv")

df.to_csv(
    berenslab_data_path / "pubmed_baseline_2025.zip",
    index=False,
    compression=compression_opts,
)

CPU times: user 35min 29s, sys: 24.3 s, total: 35min 53s
Wall time: 36min 7s
